In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|█████████████████| 28.0/28.0 [00:00<00:00, 78.3kB/s]
config.json: 100%|█████████████████████████████| 570/570 [00:00<00:00, 1.21MB/s]
vocab.txt: 100%|██████████████████████████████| 232k/232k [00:00<00:00, 411kB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 1.45MB/s]


In [3]:
raw_datasets = load_dataset("glue","mrpc")
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
